Hyrbid Importance Sampling with Markov Chains For Monte Crlo Integration

In [9]:
import nbimporter

from Importance_Int import importance_integration, target_pdf

#==============================Dependencies================================

#libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, truncnorm
from numpy.random import Generator, MT19937
import time
from scipy.special import logsumexp

#Initializing the random number generator
seed = int(time.time()) 
bitgen = MT19937(seed)
rng = Generator(bitgen)  # reproducible generator

In [10]:
#=======================Metropolis-Hastings Algorithm (Python)========================
def metropolis_python(f, N, x0, step_size, burnin, thinning, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    total_steps = N * thinning + burnin
    samples = []
    x = x0
    count = 0

    for i in range(total_steps):
        x_cand = x + rng.uniform(-step_size, step_size)
        alpha = min(1.0, abs(target_pdf(x_cand) / target_pdf(x)))
        if rng.uniform() < alpha:
            x = x_cand
            count += 1
        if i >= burnin and (i - burnin) % thinning == 0:
            samples.append(x)
    acceptance_rate = count / total_steps
    return np.array(samples)

In [11]:
#=======================Kernel Density Estimation (KDE) Functions========================

# Fixed-bandwidth KDE
def pilot_kde(samples, bandwidth): #fixed KDE mthod
    """Fixed-bandwidth KDE using Gaussian kernel."""
    def kde_eval(x_eval):
        x_eval = np.atleast_1d(x_eval)
        n = len(samples)
        coeff = 1 / (n * bandwidth * np.sqrt(2 * np.pi))
        diffs = (x_eval[:, None] - samples[None, :]) / bandwidth
        return coeff * np.sum(np.exp(-0.5 * diffs**2), axis=1)
    return kde_eval

def silverman_bandwidth(samples):
    n = len(samples)
    std = np.std(samples, ddof=1)
    return 1.06 * std * n ** (-1/5)

# Adaptive-bandwidth KDE
def adaptive_kde(samples, h_fixed, alpha=0.5): # Adative KDE method
    """
    Adaptive KDE using Abramson's square-root law.
    samples : 1D array of data points
    h_fixed : base bandwidth for pilot KDE
    alpha   : sensitivity parameter (default 0.5)
    """
    n = len(samples)

    # Step 1: pilot density estimate
    pilot = pilot_kde(samples, h_fixed)
    f_i = pilot(samples)

    # Step 2: compute geometric mean of pilot estimates
    g = np.exp(np.mean(np.log(f_i)))

    # Step 3: compute local bandwidth factors
    lambda_i = (f_i / g)**(-alpha)
    h_i = h_fixed * lambda_i

    # Step 4: adaptive KDE function
    def kde_adaptive(x_eval):
        x_eval = np.atleast_1d(x_eval)
        coeffs = 1 / (np.sqrt(2 * np.pi) * h_i)
        diffs = (x_eval[:, None] - samples[None, :]) / h_i
        result = np.sum(coeffs * np.exp(-0.5 * diffs**2), axis=1) / n
        return result

    return kde_adaptive

In [12]:
def hybrid_importance_sampling(f, m_chain, target_pdf):
    """
    Estimate the integral of f using importance sampling from KDE-estimated proposal.

    Returns:
        estimate: Monte Carlo estimate of the integral
        stderr: Standard error of the estimate (not just std of weights)
    """
    weights = f(m_chain) / target_pdf(m_chain)
    estimate = np.mean(weights)

    return estimate


In [13]:
#==========================Gaussian Target Distribution==========================
def target_pdf(x, mu=0, sigma=1.2):
    """
    Compute the PDF of a Gaussian distribution with mean 0 and standard deviation 1.
    """
    """Probability-density function of N(mu, sigma^2)."""
    '''z = (x - mu) / sigma
    return np.exp(-0.5 * z**2) / (sigma * np.sqrt(2 * np.pi)) #norm.pdf(x, mu, sigma)'''

    return (1/np.pi)*(1/(1+x**2)) # Cauchy distribution

In [14]:
def f(x):
    return np.exp(-x**2)

step_size=2
N_list = np.logspace(1, 4, num=50, dtype=int)

meta_hybrid_results = []
meta_importance_results = []
hybrid_stderr = []  
importance_stderr = []


for i in range(15):
    hybrid_results = []
    importance_results = [] 
    for N in N_list:
        m_chain_hybrid = metropolis_python(f, N, rng.uniform(-10, 10), step_size, 2000, 10)
        h_fixed = silverman_bandwidth(m_chain_hybrid)
        kde = adaptive_kde(m_chain_hybrid, h_fixed, alpha=0.5)
        hybrid_estimate = hybrid_importance_sampling(f, m_chain_hybrid, kde)
        m_chain_importance = metropolis_python(target_pdf, N, rng.uniform(-10, 10), step_size, 2000, 10)
        importance_estimate = hybrid_importance_sampling(f, m_chain_importance, target_pdf)
        hybrid_results.append(hybrid_estimate)
        importance_results.append(importance_estimate)


    hybrid_results = np.array(hybrid_results)
    importance_results = np.array(importance_results)
    meta_hybrid_results.append(hybrid_results)
    meta_importance_results.append(importance_results)

# Convert lists to NumPy arrays
meta_hybrid_results = np.array(meta_hybrid_results)
meta_importance_results = np.array(meta_importance_results)

# Calculate the standard error element-wise for the columns
hybrid_stderr = np.std(meta_hybrid_results, axis=0, ddof=1) / np.sqrt(meta_hybrid_results.shape[0])
importance_stderr = np.std(meta_importance_results, axis=0, ddof=1) / np.sqrt(meta_importance_results.shape[0])

# Calculate the average element-wise in the column direction for both methods
hybrid_avg = np.mean(meta_hybrid_results, axis=0)
importance_avg = np.mean(meta_importance_results, axis=0)



In [15]:
plt.figure(figsize=(15, 7))
plt.rcParams.update({'axes.titlesize': 16, 'axes.titleweight': 'bold', 'axes.titlepad': -20})
plt.errorbar(N_list, hybrid_avg, yerr=hybrid_stderr, fmt='-s', capsize=5, label='Hybrid MCMC + KDE, alpha=0.5 (with stderr)', linewidth=1.2, alpha=1)
plt.errorbar(N_list, importance_avg, yerr=importance_stderr, fmt='-s', capsize=5, label='Importance MCMC (with stderr)', linewidth=1.2, alpha=1)
# Plot estimates with error bars
analytical_result = np.pi
plt.axhline(y=analytical_result, color='red', linestyle='--', label=r'Analytical Result = $\sqrt(\pi)$', alpha=0.5)
plt.xscale('log')
plt.xlabel('Number of Samples (N)')
plt.ylabel('Value')
function_description = r"$\int_{-\infty}^{\infty}e^{-x^2}dx$"  # Update the function description with infinity symbol
plt.title(f'Monte Carlo Integral Estimate vs N\nFunction: {function_description}', fontsize=16)
plt.legend(fontsize=12, loc='best')
plt.grid(True)
plt.savefig(f"plots/'MCMCKDE_vs_MCMC_IMP{function_description}_step={step_size}.pdf", format="pdf")
plt.show()

ValueError: 
\sqrt(\pi)
     ^
Expected \sqrt{value}, found '('  (at char 5), (line:1, col:6)

ValueError: 
\sqrt(\pi)
     ^
Expected \sqrt{value}, found '('  (at char 5), (line:1, col:6)

<Figure size 1080x504 with 1 Axes>

In [ ]:
'''#========================Example Usage of Hybrid Importance Sampling========================
N=10000
m_chain = metropolis_python(f, N, rng.uniform(-10, 10), 6, 2000, 10)
h_fixed = silverman_bandwidth(m_chain)
kde = adaptive_kde(m_chain, h_fixed, alpha=0.05)

plt.plot(m_chain, kde(m_chain), 'o', markersize=1, label='KDE Samples')
plt.show()

print("Hybrid Importance Sampling Estimate:"),
hybrid_importance_sampling(f, m_chain, kde)'''

'#========================Example Usage of Hybrid Importance Sampling========================\nN=10000\nm_chain = metropolis_python(f, N, rng.uniform(-10, 10), 6, 2000, 10)\nh_fixed = silverman_bandwidth(m_chain)\nkde = adaptive_kde(m_chain, h_fixed, alpha=0.05)\n\nplt.plot(m_chain, kde(m_chain), \'o\', markersize=1, label=\'KDE Samples\')\nplt.show()\n\nprint("Hybrid Importance Sampling Estimate:"),\nhybrid_importance_sampling(f, m_chain, kde)'

In [ ]:
'''# Generate a Markov chain using the Metropolis-Hastings algorithm
N = 10000
plt.figure(figsize=(10, 6))
for i in range(50):
    m_chain = metropolis_python(f, N, 0, 3, 0, 1)

    # Plot the Markov chain against its length
    #plt.figure(figsize=(10, 6))
    plt.plot(range(len(m_chain)), m_chain, label="Markov Chain", linewidth=0.85)
plt.xlabel("Step")
plt.ylabel("Value")
plt.title("Markov Chains")
#plt.legend()
plt.grid()
plt.savefig("Plots/markov_chain_plot_fast_mix.pdf")
plt.show()'''

'# Generate a Markov chain using the Metropolis-Hastings algorithm\nN = 10000\nplt.figure(figsize=(10, 6))\nfor i in range(50):\n    m_chain = metropolis_python(f, N, 0, 3, 0, 1)\n\n    # Plot the Markov chain against its length\n    #plt.figure(figsize=(10, 6))\n    plt.plot(range(len(m_chain)), m_chain, label="Markov Chain", linewidth=0.85)\nplt.xlabel("Step")\nplt.ylabel("Value")\nplt.title("Markov Chains")\n#plt.legend()\nplt.grid()\nplt.savefig("Plots/markov_chain_plot_fast_mix.pdf")\nplt.show()'

In [ ]:
'''# Generate bimodal data
data = np.concatenate([rng.normal(loc=-2, scale=0.8, size=5000), rng.normal(loc=3, scale=1.2, size=5000)])

# Compute KDE using adaptive KDE
h_fixed_bimodal = silverman_bandwidth(data)
kde_bimodal = adaptive_kde(data, h_fixed_bimodal, alpha=0.5)

# Generate evaluation points for KDE
x_eval = np.linspace(data.min() - 1, data.max() + 1, 1000)
kde_values = kde_bimodal(x_eval)

# Plot histogram and KDE
plt.figure(figsize=(10, 6))
plt.hist(data, bins=50, density=True, alpha=0.6, label='Histogram')
plt.plot(x_eval, kde_values, color='red', linewidth=2, label='KDE (Adaptive),  Alpha=0.5')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Bimodal Distribution: Histogram and KDE')
plt.legend()
plt.grid(True)
plt.savefig("plots/bimodal_distribution_histogram_kde.pdf", format="pdf")
plt.show()'''

'# Generate bimodal data\ndata = np.concatenate([rng.normal(loc=-2, scale=0.8, size=5000), rng.normal(loc=3, scale=1.2, size=5000)])\n\n# Compute KDE using adaptive KDE\nh_fixed_bimodal = silverman_bandwidth(data)\nkde_bimodal = adaptive_kde(data, h_fixed_bimodal, alpha=0.5)\n\n# Generate evaluation points for KDE\nx_eval = np.linspace(data.min() - 1, data.max() + 1, 1000)\nkde_values = kde_bimodal(x_eval)\n\n# Plot histogram and KDE\nplt.figure(figsize=(10, 6))\nplt.hist(data, bins=50, density=True, alpha=0.6, label=\'Histogram\')\nplt.plot(x_eval, kde_values, color=\'red\', linewidth=2, label=\'KDE (Adaptive),  Alpha=0.5\')\nplt.xlabel(\'Value\')\nplt.ylabel(\'Density\')\nplt.title(\'Bimodal Distribution: Histogram and KDE\')\nplt.legend()\nplt.grid(True)\nplt.savefig("plots/bimodal_distribution_histogram_kde.pdf", format="pdf")\nplt.show()'